
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChanifRusydi/simple_chatbot/blob/main/chatbot_with_tensorflow.ipynb

In [ ]:
!pip install nltk
!pip install tensorflow-gpu

In [ ]:
from nltk.stem import WordNetLemmatize
import json
import pickle
import numpy as np
import tensorflow as tf


In [ ]:
ignore_symbols=['?','!']
intents_json=json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]

for intent in intents_json['intents']:
    for j in intent['tags']:
        word=nltk.word_tokenize(j)
        words.extend(word)
        documents.append((word,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words=[lemmetizer.lemmatize(w.lower()) for w in words if w not in ignore_symbols]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))

In [ ]:
training=[]
output=[0]*len(classes)
for document in  documents:
    word=[]
    pattern=document[0]
    pattern=[lemmetizer.lemmatize(w.lower()) for w in pattern]
    for w in words:
        word.append(1 if w in pattern else 0)
    output_row=list(output)
    output_row[classes.index(document[1])]=1
    training.append([word,output_row])

In [ ]:
training=np.array(training)
train_x=training[:,0]
train_y=training[:,1]
train_x

In [ ]:
train_y

In [ ]:
model=tf.keras.model.Sequential()
model.add(tf.keras.layers.Embedding(len(words),8))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
history=model.fit(np.array(training),epochs=200,batch_size=5,validation_split=0.1)

In [ ]:
model.save('chatbot_model.h5')